In [1]:
import numpy as np
import cnfgen
from pysat.formula import CNF
from pysat.solvers import Glucose3
import pickle
import random
import glob
from os.path import join

In [4]:
n = 10 # np.random.randint(0,1000)
print("n",n)
p = np.random.rand(n) # this is the probability to sample a 1
print("p", p)
x = np.random.rand(m)
print("x", x)
m = 50 # np.random.randint(0,2000)
print("m", m)

# matrix = np.random.randint(-1,2,size=n*m).reshape((n,m))
# print(matrix)


n 10
p [0.38640736 0.37640649 0.06802916 0.93269792 0.98583918 0.90065875
 0.07917074 0.13351898 0.10746508 0.71544212]
x [0.7239866  0.08960626 0.28253234 0.17262954 0.211082   0.15185316]
m 50


In [2]:
# definition of some functions that might come in handy...

def find_prob_matrix(matrix, p):
    p_double = np.array([p,1-p]).transpose()
    positive = abs(matrix * np.array([p_double[:,0]]).transpose() - abs(matrix) * np.array([p_double[:,0]]).transpose())/2
    negative = abs(matrix * np.array([p_double[:,1]]).transpose() + abs(matrix) * np.array([p_double[:,1]]).transpose())/2
    mask = np.where(np.sum(abs(matrix), axis = 0)!=0, 1, 0)*np.ones(matrix.shape)
    zeros = (np.ones(np.shape(matrix)) - abs(matrix)) * mask
    return negative + positive + zeros

def find_induced_probs(matrix, p):
    return np.prod(find_prob_matrix(matrix, p), axis = 0)

def find_neighbors(matrix, conflicting_only = False):
    occurencies_matrix =abs(matrix)
    neighbors_indices_list = []
    neighbors_number_list = []
    for i in range(np.shape(occurencies_matrix)[1]):
        target = matrix[:,i][:, None]

        if conflicting_only == False:
            a = np.where(np.sum(occurencies_matrix[:,0:i] * abs(target), axis = 0)!=0, 1, 0)
            b = np.where(np.sum(occurencies_matrix[:,i+1:np.shape(occurencies_matrix)[1]] * abs(target), axis = 0)!=0, 1, 0)
        if conflicting_only == True:
            a = np.where(np.sum(((occurencies_matrix[:,0:i] * abs(target)) - (matrix[:, 0:i] * target))/2, axis = 0) != 0 , 1, 0)
            b = np.where(np.sum(((occurencies_matrix[:,i+1:np.shape(occurencies_matrix)[1]] * abs(target)) - (matrix[:, i+1:np.shape(occurencies_matrix)[1]] * target))/2, axis = 0) != 0, 1, 0)
        
        a = np.hstack((a,[0]))      
        remainder = np.hstack((a, b))
        neighbors_number = np.sum(remainder)
        neighbors_indices = np.argwhere(remainder != 0).transpose()[0]
        neighbors_number_list.append(neighbors_number)
        neighbors_indices_list.append(neighbors_indices)

    return(neighbors_indices_list,neighbors_number_list)

def get_rhs(matrix, x, conflicting_only = False):
    neighbors_indices_list,neighbors_number_list = find_neighbors(matrix, conflicting_only=conflicting_only)
    m = len(neighbors_indices_list)
    rhs_array = np.zeros(m)
    for current_clause in range(m):
        rhs = x[current_clause]
        for neighbor in neighbors_indices_list[current_clause]:
            rhs = rhs * (1-x[neighbor])
        rhs_array[current_clause] = rhs
    return rhs_array

def check_uniqueness_condition(matrix):
    unique_matrix = np.unique(matrix, axis = 1)
    return ((matrix.shape[1] - np.sum(np.where(np.sum(matrix, axis= 0)!= 0, 0, 1))) - (unique_matrix.shape[1] - np.sum(np.where(np.sum(unique_matrix, axis= 0)!= 0, 0, 1))) == 0)

In [141]:
# sampling of a matrix according to p

def satisfies_lll(matrix, p, conflicting_only = False):
    d = np.max(find_neighbors(matrix, conflicting_only = conflicting_only)[1])
    # maybe change it such that only the relevant neighborhood is considered
    induced_probs = find_induced_probs(matrix, p)
    max_p = np.max(induced_probs)
    return d*max_p*np.e <=1


def satisfies_lll_masked(matrix, p, conflicting_only = False):
    d_list = find_neighbors(matrix, conflicting_only = conflicting_only)[1]
    x_list = 1/(d_list + np.ones(len(d_list)))
    rhs = get_rhs(matrix, x_list, conflicting_only=conflicting_only)
    induced_probs = find_induced_probs(matrix, p)
    return np.all(np.less_equal(induced_probs - rhs, 0))


def update_matrix_single_elements(matrix, p, t, TIMEOUT):
    available_indices = np.argwhere(matrix == 0)
    if len(available_indices) != 0:
        chosen_index = random.choice(available_indices)
        prob_pick = p[chosen_index[0]]
        old_matrix = np.copy(matrix)
        choice = np.random.choice([int(-1),int(1)], p = [1- prob_pick, prob_pick]) # p is the prob to sample 1
        matrix[chosen_index[0],chosen_index[1]] = choice

        check_LLL = satisfies_lll(matrix, p, conflicting_only=conflicting_only)
        check_uniqueness = check_uniqueness_condition(matrix)
        #if not np.all(np.greater_equal(0, induced_probs - rhs)):
        if not np.all([check_LLL, check_uniqueness]):
            #print("matrix", np.allclose(old_matrix, matrix))
            matrix = old_matrix
            t += 1
            # print("t =", t)
    else:
        t = TIMEOUT
    return matrix, t


def generate_new_clause(p, k):
    variables = sorted(np.arange(0,len(p),1))
    chosen_indices = random.sample(variables, k)
    new_clause = np.zeros(len(p))
    for index in chosen_indices:
        prob_pick = p[index]
        new_clause[index] = np.random.choice([int(-1),int(1)], p = [1- prob_pick, prob_pick])
    return new_clause


def update_matrix_clauses(matrix, p, TIMEOUT_internal, conflicting_only = False, k = False):
    t_internal = 0
    for clause_i in range(matrix.shape[1]):
        status_internal = False
        while status_internal==False and t_internal <= TIMEOUT_internal:
            old_matrix = np.copy(matrix)
            if k == False:
                k_used = np.random.randint(2, len(p)+1)
            else:
                k_used = k
            new_clause = generate_new_clause(p, k_used)
            matrix[:,clause_i] = new_clause
            check_LLL = satisfies_lll_masked(matrix, p, conflicting_only=conflicting_only)
            check_uniqueness = check_uniqueness_condition(matrix)
            if not np.all([check_LLL, check_uniqueness]):
                matrix = old_matrix
                t_internal += 1
            else: 
                status_internal = True
    return matrix


def get_matrix(p, n, m, k= False, conflicting_only = False, TIMEOUT_internal = 100, TIMEOUT = 100):
    matrix = np.zeros((n,m))
    best_matrix = np.zeros(n)[:, None]
    print(best_matrix.shape)
    t = 0
    status = False
    # while t < TIMEOUT:
        # matrix, t = update_matrix_single_elements(matrix, p, t, TIMEOUT)
    status_shape = False
    while status_shape == False and t < TIMEOUT :
        matrix = update_matrix_clauses(matrix, p, TIMEOUT_internal = TIMEOUT_internal, conflicting_only=conflicting_only, k = k)
        matrix = np.unique(matrix, axis = 1)
        cleaned_matrix = []
        for i in range(matrix.shape[1]):
            if np.sum(abs(matrix[:,i])) > 1:
                cleaned_matrix.append(matrix[:,i])
        cleaned_matrix = np.array(cleaned_matrix).T
        print(cleaned_matrix.shape)
        if cleaned_matrix.shape[1] == m:
            status_shape = True
            best_matrix = cleaned_matrix
        else:
            if cleaned_matrix.shape[1] > best_matrix.shape[1]:
                best_matrix = cleaned_matrix
            matrix = np.zeros((n,m))
            t += 1
        print(best_matrix.shape)
    d_list = find_neighbors(matrix, conflicting_only = conflicting_only)[1]
    x_list = 1/(d_list + np.ones(len(d_list)))
    print("neighbors", np.min(d_list), np.max(d_list))
    print("x_list", np.min(x_list), np.max(x_list))
    print("masked", satisfies_lll_masked(matrix, p, conflicting_only=conflicting_only))
    print("usual", satisfies_lll(matrix, p, conflicting_only=conflicting_only))
    assert satisfies_lll_masked(matrix, p, conflicting_only=conflicting_only) == True
    """matrix = np.unique(matrix, axis = 1)
    cleaned_matrix = []
    for i in range(matrix.shape[1]):
        if np.sum(abs(matrix[:,i])) > 1:
            cleaned_matrix.append(matrix[:,i])
    cleaned_matrix = np.array(cleaned_matrix).T"""
    return best_matrix

n = 10 # np.random.randint(0,1000)
print("n",n)
# p = np.ones(n)/2
p = np.random.rand(n) # this is the probability to sample a 1
print("p", p)
m = 100 # np.random.randint(0,2000)
print("m", m)




matrix = get_matrix(p, n, m, k= 8, conflicting_only=False)
print(matrix)
print(np.shape(matrix))
print(np.sum(abs(matrix), axis = 0))


"""

def cleverly_readjust_p_and_matrix(matrix,p):
    max_d, constraint_node_idx = find_max_degree_from_constraint_nodes(matrix)
    edges = edges[receiver=constraint_node_idx]
    to_drop = np.random.choose(edges,1)
    matrix = remove_edges(to_drop, matrix)
    return matrix,p

while not satisfies_lll(matrix, p):
    p, matrix = cleverly_readjust_p_and_matrix(matrix, p)

return matrix, p

"""
# above was Paul's suggestion

n 10
p [0.02673382 0.56108523 0.35321632 0.63801951 0.88492276 0.80064313
 0.19281372 0.32293195 0.08385592 0.77490693]
m 100
(10, 1)
(10, 98)
(10, 98)
(10, 99)
(10, 99)
(10, 99)
(10, 99)
(10, 98)
(10, 99)
(10, 100)
(10, 100)
neighbors 99 99
x_list 0.01 0.01
masked True
usual True
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.
  -1. -1.

'\n\ndef cleverly_readjust_p_and_matrix(matrix,p):\n    max_d, constraint_node_idx = find_max_degree_from_constraint_nodes(matrix)\n    edges = edges[receiver=constraint_node_idx]\n    to_drop = np.random.choose(edges,1)\n    matrix = remove_edges(to_drop, matrix)\n    return matrix,p\n\nwhile not satisfies_lll(matrix, p):\n    p, matrix = cleverly_readjust_p_and_matrix(matrix, p)\n\nreturn matrix, p\n\n'

In [22]:
# do an optimization of x assignment

from scipy.optimize import minimize


def check_LLL_condition(matrix, p, x, conflicting_only = False):
    induced_probs = find_induced_probs(matrix, p)
    rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
    # print(induced_probs - rhs)
    # print(np.less_equal(induced_probs - rhs, 0))
    # print(np.all(np.less_equal(induced_probs - rhs, 0)))
    return np.all(np.less_equal(induced_probs - rhs, 0))


def satisfies_lll_masked(matrix, p, conflicting_only = False):
    d_list = find_neighbors(matrix, conflicting_only = conflicting_only)[1]
    x_list = 1/(d_list + np.ones(len(d_list)))
    rhs = get_rhs(matrix, x_list, conflicting_only=conflicting_only)
    induced_probs = find_induced_probs(matrix, p)
    return np.all(np.less_equal(induced_probs - rhs, 0))


def satisfies_lll_optimization_masked(matrix, p, x_list, conflicting_only = False, maxiter = 100):
    def compute_lll_condition(x_list):
        neighbors_indices_list,neighbors_number_list = find_neighbors(matrix, conflicting_only=conflicting_only)
        m = len(neighbors_indices_list)
        rhs_array = np.zeros(m)
        for current_clause in range(m):
            rhs = x_list[current_clause]
            for neighbor in neighbors_indices_list[current_clause]:
                rhs = rhs * (1-x_list[neighbor])
            rhs_array[current_clause] = rhs
        induced_probs = find_induced_probs(matrix, p)
        objective = np.sum(np.maximum(induced_probs - rhs_array,np.zeros(len(induced_probs))))
        #objective = induced_probs - rhs_array
        #print(objective)
        #print(np.sum(np.maximum(objective, np.zeros(len(objective)))))
        return objective

    lll_condition = check_LLL_condition(matrix, p, x_list, conflicting_only = False)
    if lll_condition == False:
        x_optimized = minimize(compute_lll_condition, x_list, method='BFGS', options={'maxiter': maxiter})
        lll_condition = check_LLL_condition(matrix, p, x_optimized.x, conflicting_only=conflicting_only)
        if lll_condition :
            # print("optimized works")
            x_list = x_optimized.x
    return lll_condition, x_list


def check_LLL_condition(matrix, p, x, conflicting_only = False):
    induced_probs = find_induced_probs(matrix, p)
    rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
    # print(induced_probs - rhs)
    # print(np.less_equal(induced_probs - rhs, 0))
    # print(np.all(np.less_equal(induced_probs - rhs, 0)))
    return np.all(np.less_equal(induced_probs - rhs, 0))


def generate_new_clause(p, k):
    variables = sorted(np.arange(0,len(p),1))
    chosen_indices = random.sample(variables, k)
    new_clause = np.zeros(len(p))
    for index in chosen_indices:
        prob_pick = p[index]
        new_clause[index] = np.random.choice([int(-1),int(1)], p = [1- prob_pick, prob_pick])
    return new_clause


def update_matrix_clauses_optimized(matrix, p, TIMEOUT_internal, conflicting_only = False, k = False):
    t_internal = 0
    x_list = np.zeros(matrix.shape[1])
    for clause_i in range(matrix.shape[1]):
        print("clause", clause_i)
        status_internal = False
        while status_internal==False and t_internal <= TIMEOUT_internal:
            old_matrix = np.copy(matrix)
            if k == False:
                k_used = np.random.randint(2, len(p)+1)
            else:
                k_used = k
            new_clause = generate_new_clause(p, k_used)
            matrix[:,clause_i] = new_clause
            d = find_neighbors(matrix, conflicting_only = False)[1]
            x_list[clause_i] =  1/(1+d[clause_i])
            # maybe also change the other affected terms...
            # print("usual masked", satisfies_lll_masked(matrix, p, conflicting_only = conflicting_only))
            # print("old", x_list)
            check_LLL, x_list = satisfies_lll_optimization_masked(matrix, p, x_list, conflicting_only = conflicting_only)
            print(check_LLL, check_LLL_condition(matrix, p, x_list, conflicting_only = conflicting_only))
            #print("check_LLL", check_LLL)
            # print("optimized", x_list)
            check_uniqueness = check_uniqueness_condition(matrix)
            #print("uniqueness", check_uniqueness)
            #print([check_LLL, check_uniqueness])
            if not np.all([check_LLL, check_uniqueness]):
                matrix = old_matrix
                # print("use old matrix")
                t_internal += 1
            else: 
                status_internal = True
    return matrix, x_list


def get_matrix_optimized(p, n, m, k= False, conflicting_only = False, TIMEOUT_internal = 100, TIMEOUT = 100):
    matrix = np.zeros((n,m))
    best_matrix = np.zeros(n)[:, None]
    best_matrix_x = 0
    t = 0
    status = False
    # while t < TIMEOUT:
        # matrix, t = update_matrix_single_elements(matrix, p, t, TIMEOUT)
    status_shape = False
    while status_shape == False and t < TIMEOUT :
        matrix, x_list = update_matrix_clauses_optimized(matrix, p, TIMEOUT_internal = TIMEOUT_internal, conflicting_only=conflicting_only, k = k)
        print(matrix.shape)
        # matrix = np.unique(matrix, axis = 1)
        # print(matrix)
        cleaned_matrix = []
        cleaned_x = []
        for i in range(matrix.shape[1]):
            if np.sum(abs(matrix[:,i])) > 1:
                cleaned_matrix.append(matrix[:,i])
                cleaned_x.append(x_list[i])
        cleaned_matrix = np.array(cleaned_matrix).T
        print(cleaned_matrix.shape)
        if cleaned_matrix.shape[1] == m:
            status_shape = True
            best_matrix = cleaned_matrix
            best_matrix_x = cleaned_x
        else:
            if cleaned_matrix.shape[1] > best_matrix.shape[1]:
                best_matrix = cleaned_matrix
                best_matrix_x = cleaned_x
            matrix = np.zeros((n,m))
            t += 1
        print(best_matrix.shape)
    #d_list = find_neighbors(matrix, conflicting_only = conflicting_only)[1]
    #x_list = 1/(d_list + np.ones(len(d_list)))
    #print("neighbors", np.min(d_list), np.max(d_list))
    print("x_list", np.min(best_matrix_x), np.max(best_matrix_x))
    print("optimized", check_LLL_condition(best_matrix, p, best_matrix_x, conflicting_only = conflicting_only))
    print("masked", satisfies_lll_masked(best_matrix, p, conflicting_only=conflicting_only))
    assert check_LLL_condition(best_matrix, p, best_matrix_x, conflicting_only = conflicting_only) == True
    return best_matrix

n = 8 # np.random.randint(0,1000)
print("n",n)
# p = np.ones(n)/2
p = np.random.rand(n) # this is the probability to sample a 1
print("p", p)
m = 70 # np.random.randint(0,2000)
print("m", m)




# matrix = get_matrix_optimized(p, n, m, k= 5, conflicting_only = False)
matrix = get_matrix_optimized(p, n, m, k= 5, conflicting_only = False)
print(matrix)
print(np.shape(matrix))
print(np.sum(abs(matrix), axis = 0))
# above was Paul's suggestion

n 8
p [0.72845471 0.46758845 0.27150121 0.05196965 0.68615648 0.64188514
 0.69155047 0.98255363]
m 70
clause 0
True True
clause 1
True True
clause 2
True True
clause 3
True True
clause 4
True True
clause 5
True True
clause 6
True True
clause 7
True True
clause 8
True True
clause 9
True True
clause 10
True True
clause 11
True True
clause 12
True True
clause 13
True True
clause 14
True True
clause 15
True True
clause 16
True True
clause 17
True True
clause 18
True True
clause 19
True True
True True
clause 20
True True
clause 21
True True
clause 22
True True
clause 23
True True
clause 24
True True
clause 25
True True
clause 26
True True
clause 27
True True
clause 28
True True
clause 29
True True
clause 30
True True
clause 31
True True
clause 32
True True
clause 33
True True
clause 34
True True
True True
clause 35
True True
clause 36
True True
clause 37
True True
clause 38
True True
clause 39
True True
clause 40
True True
clause 41
True True
clause 42
True True
clause 43
True True
clause 4

In [23]:
print(matrix)

[[ 1. -1. -1.  1.  1.  1.  0.  1.  0.  1.  1. -1.  1.  0.  1.  1.  1. -1.
   0.  1.  1.  0.  1.  1.  0.  0. -1.  0.  1.  1.  1.  1.  0.  0.  1.  1.
   0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.  1.  1.  1.  0.  1.  1.  1.
   1. -1.  0.  0.  0.  0. -1.  1.  0.  1.  0.  1.  1. -1.  0.  1.]
 [ 1.  0.  0.  1.  1.  0.  1.  1.  1.  1.  0. -1.  1. -1. -1.  0.  0.  1.
   1.  0.  0.  1.  0.  0. -1. -1. -1.  1.  0.  1.  1.  1. -1.  1.  1. -1.
   0.  0. -1. -1.  0.  1. -1.  0.  1.  1.  1.  0.  1. -1. -1.  0.  0. -1.
   0. -1. -1.  0.  0. -1.  1.  1. -1.  0.  1.  1.  0. -1.  1.  0.]
 [-1.  0.  1. -1.  0.  0.  1.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  1.  1.  0.  1. -1. -1. -1. -1.  0. -1. -1. -1.  1.  0. -1. -1.  0.
   1.  0. -1.  0.  1.  1. -1. -1.  0. -1.  0. -1. -1. -1.  0.  0. -1.  0.
   1.  0.  0. -1.  1.  1.  0.  0. -1.  1.  1.  0. -1. -1.  1. -1.]
 [-1. -1. -1.  0. -1. -1. -1. -1.  0.  0. -1. -1. -1. -1. -1.  1. -1.  0.
   0.  0. -1. -1.  0.  0. -1.  0.  0. -1. -1. -1. -1.  0. -

In [119]:
# sampling of a matrix according to p, x (currently not giving results that are useful...)

def check_LLL_condition(matrix, p, x, conflicting_only = False):
    induced_probs = find_induced_probs(matrix, p)
    rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
    # print(induced_probs - rhs)
    # print(np.less_equal(induced_probs - rhs, 0))
    # print(np.all(np.less_equal(induced_probs - rhs, 0)))
    return np.all(np.less_equal(induced_probs - rhs, 0))

def get_LLL_matrix(p, x, conflicting_only = False, TIMEOUT = 1000):
    n = len(p)
    m = len(x)
    matrix = np.zeros((n,m))
    t = 0
    while t < TIMEOUT:
        available_indices = np.argwhere(matrix == 0)
        if len(available_indices) != 0:
            chosen_index = random.choice(available_indices)
            prob_pick = p[chosen_index[0]]
            old_matrix = np.copy(matrix)
            choice = np.random.choice([int(-1),int(1)], p = [1- prob_pick, prob_pick]) # p is the prob to sample 1
            matrix[chosen_index[0],chosen_index[1]] = choice
            #induced_probs = find_induced_probs(matrix, p)
            #rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
            check_LLL = check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only)
            check_uniqueness = check_uniqueness_condition(matrix)
            #if not np.all(np.greater_equal(0, induced_probs - rhs)):
            if not np.all([check_LLL, check_uniqueness]):
                #print("matrix", np.allclose(old_matrix, matrix))
                matrix = old_matrix
                t += 1
                # print("t =", t)
            
        else:
            t = TIMEOUT
        # print(check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only))
    assert check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only) == True
    return np.unique(matrix, axis = 1)

matrix = get_LLL_matrix(p, x, conflicting_only= False, TIMEOUT= 1000)
print(matrix)
print(np.shape(matrix))

[[ 0.  1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0. -1.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0. -1.]]
(20, 2)


In [ ]:
# define a translation from matrix to CNF
from pysat.formula import CNF

def translate_matrix_to_CNF(matrix):
    cnf = CNF()
    for i in range(matrix.shape[1]):
        print(matrix[:, i])




In [ ]:
matrix = np.random.randint(-1,2,size=n*m).reshape((n,m))

In [ ]:
# idea from Paul:

def satisfies_lll(matrix, p):
    d = find_max_number_of_neighbors(matrix)
    induced_constraint_violation_probs = find_induced_probs(matrix, p)
    max_p = np.max(induced...)
    return d*max_p*np.e <=1


def cleverly_readjust_p_and_matrix(matrix,p):
    max_d, constraint_node_idx = find_max_degree_from_constraint_nodes(matrix)
    edges = edges[receiver=constraint_node_idx]
    to_drop = np.random.choose(edges,1)
    matrix = remove_edges(to_drop, matrix)
    return matrix,p

while not satisfies_lll(matrix, p):
    p, matrix = cleverly_readjust_p_and_matrix(matrix, p)

return matrix, p